## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from dateutil import parser
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import math

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

from sklearn import preprocessing

## Загрузка датасета

In [19]:
supermarket_val_target = pd.read_csv('supermarket_val_target.csv')
supermarket_val_target

,receipt_id,item_id,good,brand
0,13014464124,112697,NaN,аист
1,13014794207,109737,паштет,лыткаринский мпз
2,13014794207,109737,паштет,лыткаринский мпз
3,13014886074,108876,напиток,от винта
4,13015093794,103625,колбаса,славянский
...,...,...,...,...
59638,10115735495,101972,грудинка,фермерское подворье
59639,10115958556,107737,молоко,зеленые фермы
59640,10116494618,113985,сметана,борисов
59641,10117228760,115889,хлеб,пореченскимй


In [18]:
supermarket_val = pd.read_csv('supermarket_val.csv')
supermarket_val

,device_id,receipt_id,item_id,server_date,local_date,name,price,quantity,my_ckecker,good,brand
0,352398082091853,9127158637,113185,2021-09-01 09:11:34,2021-09-01 09:11:32,Сельдь с/с,159.00,0.42,NaN,сельдь,NaN
1,352398082091853,9127158637,106884,2021-09-01 09:11:34,2021-09-01 09:11:32,Майонез Прасковья Молочкова Провансаль 51% 400...,98.00,1.00,NaN,майонез,прасковья молочкова
2,352398080037759,9127241099,110102,2021-09-01 09:18:10,2021-09-01 09:18:09,Печенье Брянская изюминка эконом /Брянконфи/,250.00,0.28,NaN,печенье,брянконфи
3,352398080037759,9127241099,108848,2021-09-01 09:18:10,2021-09-01 09:18:09,Напиток б/а Мастер 1.5л кола ПЭТ,43.00,1.00,NaN,напиток,мастер
4,352398080037759,9127241099,117146,2021-09-01 09:18:10,2021-09-01 09:18:09,Яйцо куриное столовое 1кат /Птицефабрика Борки/,73.00,1.00,NaN,яйцо,борки
...,...,...,...,...,...,...,...,...,...,...,...
172653,352398080098702,16209456731,115873,2023-08-31 18:27:23,2023-08-31 18:27:20,Хлеб Дарницкий 600гр нарезка м/уп. /Пореченски...,46.00,1.00,NaN,хлеб,пореченский
172654,352398080037759,16209470290,109861,2023-08-31 18:28:26,2023-08-31 18:28:24,Пельмени Русские зам. /ПО Переслегино/,105.00,1.00,NaN,пельмени,переслегино
172655,352398080037759,16209470290,108943,2023-08-31 18:28:26,2023-08-31 18:28:24,Напиток квасной Особый 1.5л Вкусно среднегаз. ...,63.00,1.00,NaN,напиток,новосокольнический хлеб
172656,352398080550058,16210075242,116300,2023-08-31 19:21:01,2023-08-31 19:21:01,Чеснок /Китай/,107.64,1.00,NaN,чеснок,NaN


In [15]:
supermarket_train = pd.read_csv('supermarket_train.csv')
supermarket_train

,device_id,receipt_id,item_id,server_date,local_date,name,price,quantity,my_ckecker,good,brand
0,352398080037759,9127014611,106012,2021-09-01 08:58:39,2021-09-01 08:58:36,Краска д/волос Fitocolor 9.3 жемчужный блондин...,85.0,1.0,NaN,краска,фитокосметик
1,352398080037759,9127014611,104817,2021-09-01 08:58:39,2021-09-01 08:58:36,Картофель,129.0,1.0,NaN,картофель,NaN
2,352398080037759,9127023800,113733,2021-09-01 08:59:31,2021-09-01 08:59:31,Сиг-ты Philip Morris Blue МТ,129.0,1.0,NaN,сигареты,philip morris
3,352398080037759,9127023800,113733,2021-09-01 08:59:31,2021-09-01 08:59:31,Сиг-ты Philip Morris Blue МТ,129.0,1.0,NaN,сигареты,philip morris
4,352398080037759,9127039146,112112,2021-09-01 09:00:58,2021-09-01 09:00:58,Пряники Завируха с вишневым вкусом /телевизор/...,185.0,0.5,NaN,пряники,завируха
...,...,...,...,...,...,...,...,...,...,...,...
696782,352398080550058,16210345626,111468,2023-08-31 19:48:28,2023-08-31 19:48:28,Питьевая вода Льста 0.5л газ. ПЭТ /Бежаницкое ...,30.0,1.0,NaN,вода,льста
696783,352398080550058,16210345626,115544,2023-08-31 19:48:28,2023-08-31 19:48:28,Тягач с ракетой Тополь М фрикц. под блистером ...,270.0,1.0,NaN,тягач,NaN
696784,352398080550058,16210345626,111435,2023-08-31 19:48:28,2023-08-31 19:48:28,Пистолет Мыльные пузыри Динозавр 19см 1881/К,290.0,1.0,NaN,пистолет,NaN
696785,352398080550058,16210345626,109265,2023-08-31 19:48:28,2023-08-31 19:48:28,Нож Трамонтина №4 дерев.ручка /Китай/,140.0,1.0,NaN,нож,NaN


In [13]:
cosmetic_val_target = pd.read_csv('cosmetic_val_target.csv')
cosmetic_val_target

,receipt_id,item_id,good,brand
0,14116991683,200423,помада,letique makeup
1,14129413683,200232,крем,vanilla ecstasy
2,14129776014,200613,мусс,letique
3,14139096551,200217,крем,кедр
4,14147506937,200566,соль,бразилиан
...,...,...,...,...
22756,16191998610,200605,сыворотка,letique
22757,16191998643,200008,соль,magnesium
22758,16191998645,200247,лосьон,letique
22759,16194464879,200266,маска,letique


In [2]:
cosmetic_train = pd.read_csv('cosmetic_train.csv') # , sep='\t'
cosmetic_train

,device_id,receipt_id,item_id,local_date,name,price,quantity,good,brand
0,352398085964585,9127895765,200388,2021-09-01 10:08:23,Пакет ПВД,0.0,4,пакет,NaN
1,352398085964585,9127895765,200389,2021-09-01 10:08:23,Пакет бумажный,0.0,6,пакет,NaN
2,352398089986709,9128816187,200143,2021-09-01 11:10:58,"Коробка подарочная ""Letique""",0.0,1,коробка,letique
3,352398089986709,9128816187,200593,2021-09-01 11:10:58,"Сыворотка антиоксидантная для лица ""ANTI-POLLU...",1190.0,1,сыворотка,NaN
4,352398089986709,9128816187,200389,2021-09-01 11:10:58,Пакет бумажный,0.0,1,пакет,NaN
...,...,...,...,...,...,...,...,...,...
223903,356645110653351,16210957660,200572,2023-08-31 21:48:32,Спрей для волос для прикорневого объема волос ...,237.0,1,спрей,NaN
223904,356645110653351,16210957660,200286,2023-08-31 21:48:32,"Матирующий флюид для лица, 50 мл",417.0,2,флюид,NaN
223905,356645110653351,16210957660,200095,2023-08-31 21:48:32,Гидрогелевая маска для лица с эффектом лифтинг...,399.0,1,маска,letique
223906,356645111612489,16210996556,200525,2023-08-31 22:05:02,"Скраб для тела VANILLA ECSTASY, 250 г",0.0,1,скраб,vanilla ecstasy


In [12]:
cosmetic_val = pd.read_csv('cosmetic_val.csv')
cosmetic_val

,device_id,receipt_id,item_id,local_date,name,price,quantity,good,brand
0,352398090018898,9131497060,200555,2021-09-01 14:05:09,"Скраб лайм-имбирь, 250 г (товар)",900.0,1,скраб,NaN
1,352398090018898,9132194046,200454,2021-09-01 14:56:09,"САШЕ Обертывание-гель ""Жар и Холод"" 10мл",0.0,1,гель,sashe
2,352398090018898,9132194046,200222,2021-09-01 14:56:09,"Крем-баттер для тела Франжипани-Монои, 200 мл",1190.0,1,крем,frajipani-monoi
3,352398085964585,9132222056,200040,2021-09-01 14:58:16,"Бальзам для губ ""Сладкая клубника""",490.0,1,бальзам,NaN
4,352398085964585,9132222056,200581,2021-09-01 14:58:16,"Спрей кератиновый, 150 мл",950.0,1,спрей,NaN
...,...,...,...,...,...,...,...,...,...
51645,356645111612489,16210995964,200478,2023-08-31 22:04:39,"Скраб вишня, 250 г (товар)",357.0,2,скраб,NaN
51646,356645111612489,16210995964,200410,2023-08-31 22:04:39,Пенка для умывания с юдзу и феруловой кислотой...,417.0,2,пенка,perfection
51647,356645111612489,16210995964,200525,2023-08-31 22:04:39,"Скраб для тела VANILLA ECSTASY, 250 г",417.0,1,скраб,vanilla ecstasy
51648,356645111612489,16210995964,200259,2023-08-31 22:04:39,Маска для губ МОЛОЧНЫЙ ШОКОЛАД LIP MASK MILKY ...,327.0,2,маска,NaN


## Функции для работы с датасетом

In [6]:
def convert_dataset(_train_dataset, _val_dataset, val_data_target, key):
    train_dataset, val_dataset = {}, {}
    
    for receipt_id, item_id in zip(_train_dataset['receipt_id'], _train_dataset[key]):
        if receipt_id not in train_dataset:
            train_dataset[receipt_id] = []

        train_dataset[receipt_id].append(int(item_id))

    for receipt_id, item_id in zip(_val_dataset['receipt_id'], _val_dataset[key]):
        if receipt_id not in val_dataset:
            val_dataset[receipt_id] = []

        val_dataset[receipt_id].append(int(item_id))

    for receipt_id in val_dataset:
        val_dataset[receipt_id].append(int(val_data_target[val_data_target['receipt_id'] == receipt_id][key].tolist()[0]))

    val_keys = val_dataset.keys()
    train_dataset, val_dataset = list(train_dataset.values()), list(val_dataset.values())

    for i in range(len(train_dataset)):
        train_dataset[i] = train_dataset[i][-6:]
        
        for _ in range(6 - len(train_dataset[i])):
            train_dataset[i].insert(0, -1)

        for j in range(len(train_dataset[i])):
            train_dataset[i][j] += 1

    for i in range(len(val_dataset)):
        val_dataset[i] = val_dataset[i][-6:]
        
        for _ in range(6 - len(val_dataset[i])):
            val_dataset[i].insert(0, -1)

        for j in range(len(val_dataset[i])):
            val_dataset[i][j] += 1

    return train_dataset, val_dataset, val_keys

In [7]:
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()

In [8]:
def create_dataset(dataset, val_data, val_data_target):
    unique_train_item_ids = dataset['item_id'].unique()
    unique_train_good = dataset['good'].unique()
    unique_train_brand = dataset['brand'].unique()
    
    le1.fit(dataset['item_id'])
    dataset['item_id'] = le1.transform(dataset['item_id'])

    le2.fit(dataset['good'])
    dataset['good'] = le2.transform(dataset['good'])

    le3.fit(dataset['brand'])
    dataset['brand'] = le3.transform(dataset['brand'])
    
    val_data['item_id'] = [le1.transform([i])[0] if i in unique_train_item_ids else len(unique_train_item_ids) for i in val_data['item_id']]
    val_data_target['item_id'] = [le1.transform([i])[0] if i in unique_train_item_ids else len(unique_train_item_ids) for i in val_data_target['item_id']]
    
    val_data['good'] = [le2.transform([i])[0] if i in unique_train_good else len(unique_train_good) for i in val_data['good']]
    val_data_target['good'] = [le2.transform([i])[0] if i in unique_train_good else len(unique_train_good) for i in val_data_target['good']]
    
    val_data['brand'] = [le3.transform([i])[0] if i in unique_train_brand else len(unique_train_brand) for i in val_data['brand']]
    val_data_target['brand'] = [le3.transform([i])[0] if i in unique_train_brand else len(unique_train_brand) for i in val_data_target['brand']]

    try:
        dataset['item_id'][np.random.choice(list(range(len(dataset))), int(len(val_data['item_id']) * val_data['item_id'].value_counts()[len(unique_train_item_ids)] / len(val_data)))] = len(unique_train_item_ids)
    except:
        ...
    
    dataset = dataset.sort_values(by=['local_date'], key=lambda x: x.apply(lambda y: parser.parse(y).timestamp()))
    val_data = val_data.sort_values(by=['local_date'], key=lambda x: x.apply(lambda y: parser.parse(y).timestamp()))

    train_dataset_item_id, val_dataset_item_id, val_keys = convert_dataset(dataset, val_data, val_data_target, 'item_id')
    train_dataset_good, val_dataset_good, _ = convert_dataset(dataset, val_data, val_data_target, 'good')
    train_dataset_brand, val_dataset_brand, _ = convert_dataset(dataset, val_data, val_data_target, 'brand')

    train_dataset = [[a, b, c] for a, b, c in zip(train_dataset_item_id, train_dataset_good, train_dataset_brand)]
    val_dataset = [[a, b, c] for a, b, c in zip(val_dataset_item_id, val_dataset_good, val_dataset_brand)]

    return train_dataset, val_dataset, val_keys

## Архитектура модели

In [ ]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):
    def __init__(self, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        
        self.pos_encoder1 = PositionalEncoding(d_model, dropout)
        encoder_layers1 = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder1 = TransformerEncoder(encoder_layers1, nlayers)
        self.d_model1 = d_model
        self.linear1 = nn.Linear(d_model, ntoken1)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        
        self.embedding1.weight.data.uniform_(-initrange, initrange)
        self.linear1.bias.data.zero_()
        self.linear1.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, good: Tensor, brand: Tensor, mask: Tensor = None) -> Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            good: Tensor, shape ``[seq_len, batch_size]``
            brand: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[batch_size, ntoken]``
        """
        src = self.embedding1(src) * math.sqrt(self.d_model1)
        src = self.pos_encoder1(src)
        src_output = self.transformer_encoder1(src, mask)
        src_output = self.linear1(src_output.mean(0))

        output = torch.stack([src_output, good_output, brand_output])
        
        return output

In [22]:
device = 'cuda'

In [37]:
import pickle

# with open('datasets_cosmetic.pkl', 'wb') as w:
#     pickle.dump([train_dataset, val_dataset, list(val_keys)], w)

with open('datasets_cosmetic.pkl', 'rb') as r:
    train_dataset, val_dataset, val_keys = pickle.load(r)

## Инициализация параметров модели

In [32]:
ntokens1 = max([max(i[0]) for i in val_dataset]) + 1  # size of vocabulary supermarket cosmetic
ntokens2 = max([max(i[1]) for i in val_dataset]) + 1
ntokens3 = max([max(i[2]) for i in val_dataset]) + 1
emsize = 128  # embedding dimension
d_hid = 128  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 1  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 1  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens1, ntokens2, ntokens3, emsize, nhead, d_hid, nlayers, dropout).to(device)
model.to(device).train()

TransformerModel(
  (pos_encoder1): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder1): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=128, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (embedding1): Embedding(669, 128)
  (linear1): Linear(in_features=128, out_features=669, bias=True)
  (pos_encoder2): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3)

In [34]:
best_score = -10e100
best_epoch = 0
min_loss = 10e100
loss_history = []
metric_history = []

## Основной цикл обучения

In [35]:
for epoch in tqdm(range(100)):
    np.random.shuffle(train_dataset)
    train_x = torch.stack([torch.stack([torch.tensor(i[0][:-1]), torch.tensor(i[1][:-1]), torch.tensor(i[2][:-1])]) for i in train_dataset])
    train_y = torch.stack([torch.stack([torch.tensor(i[0][-1]), torch.tensor(i[1][-1]), torch.tensor(i[2][-1])]) for i in train_dataset])
    x_loader = DataLoader(train_x, batch_size=min(10000, train_x.shape[0]))
    y_loader = DataLoader(train_y, batch_size=min(10000, train_y.shape[0]))
    model.train()

    for x, y in zip(x_loader, y_loader):
        output = model(*x.transpose(0, 1).transpose(1, 2).to(device))

        loss = criterion(output.transpose(0, 1).transpose(1, 2), y.to(device))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        del x, y
        torch.cuda.empty_cache()
        
        current_loss = round(loss.item(), 4)
        loss_history.append(current_loss)
        
        if current_loss < min_loss:
            min_loss = current_loss

        loss_data = {
            'loss_history': loss_history
        }

        dataframe = pd.DataFrame(loss_data)
        plt.ioff()
        fig, ax = plt.subplots()
        sns.lineplot(data=dataframe, ax=ax)
        plt.title(f'Net loss, min = {min_loss}')
        ax.set(xlabel='batches', ylabel='loss')
        fig.savefig('net_loss.png')
        plt.close()

    model.eval()
    val_x = torch.stack([torch.stack([torch.tensor(i[0][:-1]), torch.tensor(i[1][:-1]), torch.tensor(i[2][:-1])]) for i in val_dataset]).transpose(0, 1).transpose(1, 2)
    val_y = torch.stack([torch.tensor(i[0][-1]) for i in val_dataset])

    output = model(*val_x.to(device))
    metric_data = accuracy_score(val_y, torch.argmax(output[0], dim=1).detach().cpu().tolist())

    del val_x, val_y
    torch.cuda.empty_cache()
    
    if metric_data > best_score:
        best_score = metric_data
        best_epoch = epoch
    
    metric_history.append(metric_data)
    
    metric_data = {
        'metric_history': metric_history # [5:]
    }
    
    dataframe = pd.DataFrame(metric_data)
    plt.ioff()
    fig, ax = plt.subplots()
    sns.lineplot(data=dataframe, ax=ax)
    plt.title(f'Net metric, min = {best_score}')
    ax.set(xlabel='batches', ylabel='metric')
    fig.savefig('net_metric.png')
    plt.close()

 22%|███████████████████████████████████████████▊                                                                                                                                                           | 22/100 [00:54<03:14,  2.49s/it]


KeyboardInterrupt: 

In [44]:
results = {}
le_dict = dict(zip(le1.transform(le1.classes_), le1.classes_))
le_dict[len(le1.classes_)] = -1
le_dict[len(le1.classes_) + 1] = -1

for pred, item_id in zip(tqdm(output[0].detach().cpu()), val_keys.copy()):
    pred = np.sort([[j, le_dict[i]] for i, j in enumerate(pred)])[::-1][:20].tolist()
    results[item_id] = pred

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22761/22761 [00:33<00:00, 674.99it/s]


In [45]:
results

{9131497060: [[-4.438217639923096, -1.0],
  [-4.33765983581543, -1.0],
  [-5.401795864105225, 200675.0],
  [-3.8037607669830322, 200674.0],
  [-5.963016986846924, 200673.0],
  [-4.4890360832214355, 200672.0],
  [-4.594630241394043, 200671.0],
  [-2.989988327026367, 200670.0],
  [-2.3070473670959473, 200669.0],
  [-5.333864688873291, 200668.0],
  [4.158067226409912, 200667.0],
  [-1.553641438484192, 200666.0],
  [-2.5705907344818115, 200665.0],
  [-5.024869918823242, 200664.0],
  [-3.2779316902160645, 200663.0],
  [-6.902379035949707, 200662.0],
  [3.033806800842285, 200661.0],
  [4.593852996826172, 200660.0],
  [-3.040518283843994, 200659.0],
  [-2.3457961082458496, 200658.0]],
 9132194046: [[-2.0291948318481445, -1.0],
  [-3.7115302085876465, -1.0],
  [-4.772855758666992, 200675.0],
  [-2.8800361156463623, 200674.0],
  [-4.494246482849121, 200673.0],
  [-2.716404438018799, 200672.0],
  [-2.586674213409424, 200671.0],
  [-2.4827053546905518, 200670.0],
  [0.6261053681373596, 200669.0],

In [46]:
with open('results.pkl', 'wb') as w:
    pickle.dump(results, w)

In [31]:
output.shape

torch.Size([3, 22761, 669])

In [32]:
y.shape

NameError: name 'y' is not defined

1. Нам дадут тест с валом или без
2. Как у других дела
3. Можно ли рассказать презентацию без 1 участника ввиду его отсутствия по семейным причинам